In [11]:
from joblib import load
import numpy as np
rf = load('models/rf_model.joblib')
kn = load('models/KN_model.joblib')
svm = load('models/svm_model.joblib')


In [12]:
from sklearn.model_selection import train_test_split

XX = np.load('X.npy', allow_pickle=True)
yy = np.load('y.npy', allow_pickle=True)

X_train, X_test, y_train, y_test = train_test_split(XX, yy, test_size=0.2, random_state=42, stratify=yy)

In [14]:
# Predict the genre for each test sample
rf_predictions = rf.predict(X_test)
svm_predictions = svm.predict(X_test)
kn_predictions = kn.predict(X_test)


In [59]:
from collections import Counter
import numpy as np

all_predictions = np.vstack([rf_predictions, svm_predictions, kn_predictions]).T

final_predictions = []
weights = [0.4, 0.4, 0.2]
weighted_predictions = []
for row in all_predictions:
    weighted_vote_counts = Counter()
    for model_prediction, weight in zip(row, weights):
        weighted_vote_counts[model_prediction] += weight

    # Choose the genre with the highest weighted vote count
    majority_vote = weighted_vote_counts.most_common(2)[0][0]
    #print(majority_vote)
    weighted_predictions.append(majority_vote)



In [60]:
from sklearn.metrics import accuracy_score

# Evaluate the ensemble's performance
accuracy = accuracy_score(y_test, weighted_predictions)
print(f'Ensemble Accuracy: {accuracy * 100:.2f}%')


Ensemble Accuracy: 51.03%


In [61]:
from datapreprosessing import * 
def classify_genre(audio_path):
    geners =[]
    features = extract_features(audio_path) 
    features = features.reshape(1, -1)
    geners.append(rf.predict(features)[0])
    geners.append(kn.predict(features)[0])
    geners.append(svm.predict(features)[0])
    return geners

d = classify_genre('fma_small/000/000002.mp3')
print(d)

['Hip-Hop', 'Hip-Hop', 'Experimental']
